# Projeto 1 - Ciência dos Dados

Nome: André Corrêa Santos
Nome: Henrique Damico

___
Carregando algumas bibliotecas:

In [147]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import nltk

In [148]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\marce\Desktop\New folder (2)


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [149]:
filename = 'base_de_dados.xlsx'

In [150]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificacao
0,quer fazer sua mulher feliz? come ela e da um ...,2
1,"@kimkataguiri que merda estão inventando, kim ...",0
2,alguém tem o link de spotify hackeado pra iphone?,0
3,@adr1nztte @charlidamelio mn pelo menos eu ain...,0
4,favela venceu e meu iphone tá vindo hj😍😘,1


In [151]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificacao
0,meu patrão quer que eu pego o iphone 12 pró ma...,0
1,comprei um m62 e vendi meu iphone me sinto com...,0
2,@omardodeboche @marcosrojerio2 @isabeledacpi e...,0
3,"alguém que tenha o iphone 7 plus, sabe me dize...",0
4,se algum amg gostasse de mim de vdd me dava um...,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o Iphone e consideramos tweets que demonstrassem a opinião popular referente ao iphone como sendo os tweets mais relevantes. Especificamente, consideramos a opinião popular quanto à características do próprio iphone, como câmera, bateria (elementos do hardware) e também caracteísticas mais subjetivas como o preço. Tweets que se distanciassem desses critérios de relevância foram avaliados como menos relevantes (evidentemente).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.








definição de funções de limpeza


In [152]:
nltk.download('stopwords') #remover stopwords
s_words = nltk.corpus.stopwords.words('portuguese')
s_list = ["de", "é", "á", "à", "ao", "a", "o", "é", "rt", '"', "“", "'", ",", "(", ")", "$", "%", "*", "&", "+", "="] #implementar stemming, palavras com mesmo stem são tratadas como mesma palavra
s_words.extend(s_list)  #remover palavras que se iniciam com @

def cleanup(lista):
        #Função de limpeza que remove caracteres nao desejados e palavras nao desejados de strings de uma lista de strings
    lista_limpa = []
    for string in lista:
        punctuation = '[!-.:?;]'
        pattern = re.compile(punctuation)   #remover pontuação
        text_subbed = re.sub(pattern, '', string)
        text_subbed.lower()
        palavras = text_subbed.split(" ") #separando a string em palavras
        for palavra in palavras:
            if palavra in s_words:      
                palavras.remove(palavra) #removendo as stopwords
            palavra.replace("\n"," ")
        lista_limpa.append(" ".join(palavras))   #transformando a lista resultante em string    

    return lista_limpa


    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


limpando a lista de opiniao, lista não opinião e lista de teste.

In [153]:

opiniao = train.loc[(train["Classificacao"] == 3) | (train["Classificacao"] == 2),:]
nao_opiniao = train.loc[(train["Classificacao"] == 0) | (train["Classificacao"] == 1),:]


texto_opiniao_limpo = cleanup(opiniao["Treinamento"].to_list())
texto_nao_opiniao_limpo = cleanup(nao_opiniao["Treinamento"].to_list())


palavras_opiniao = " ".join(texto_opiniao_limpo).split(" ")  #transformando lista de tweets limpos em uma lista de palavras contendo todas as palavras dos tweets
palavras_nao_opiniao = " ".join(texto_nao_opiniao_limpo).split(" ") 

palavras_opiniao = [w for w in palavras_opiniao if w not in s_words]
palavras_nao_opiniao = [w for w in palavras_nao_opiniao if w not in s_words]

teste = []

for tweet in test["Teste"]:
    tweet = cleanup(tweet.split(" "))
    palavras_tweet = tweet
    palavras_tweet = [w for w in palavras_tweet if w not in s_words]
    teste.append(" ".join(palavras_tweet))

teste_train = []

for tweet in train["Treinamento"]:
    tweet = cleanup(tweet.split(" "))
    palavras_tweet = tweet
    palavras_tweet = [w for w in palavras_tweet if w not in s_words]
    teste_train.append(" ".join(palavras_tweet))


pegando listas relativas e absolutas

In [154]:

lista_palavras_absoluta_opiniao = pd.Series(palavras_opiniao).value_counts()
lista_palavras_relativa_opiniao = pd.Series(palavras_opiniao).value_counts(normalize=True)

lista_palavras_absoluta_nao_opiniao = pd.Series(palavras_nao_opiniao).value_counts()
lista_palavras_relativa_nao_opiniao = pd.Series(palavras_nao_opiniao).value_counts(normalize=True)


Definindo os priors

In [155]:

total = len(lista_palavras_absoluta_opiniao)+len(lista_palavras_absoluta_nao_opiniao)
prior_op = len(lista_palavras_absoluta_opiniao)/total
prior_nop = len(lista_palavras_absoluta_nao_opiniao)/total  


VERIFICANDO O CLASSIFICADOR COM A BASE DE TESTES

In [156]:

resultado = []

for i in range(len(teste)):
    prob_op =[]
    prob_nop =[]
    for palavra in teste[i]:
        prob_dado_opiniao = 1
        prob_dado_nao_opiniao = 1

        prob_dado_opiniao *= 1/(len(lista_palavras_absoluta_opiniao)+total)
        prob_dado_nao_opiniao *= 1/(len(lista_palavras_absoluta_nao_opiniao)+total)
        if palavra in lista_palavras_absoluta_nao_opiniao:

            prob_dado_nao_opiniao += (lista_palavras_absoluta_nao_opiniao[palavra])/(len(lista_palavras_absoluta_nao_opiniao)+total)

        if palavra in lista_palavras_absoluta_opiniao:

            prob_dado_opiniao += (lista_palavras_absoluta_opiniao[palavra])/(len(lista_palavras_absoluta_opiniao)+total)

        prob_op.append(prob_dado_opiniao)
        prob_nop.append(prob_dado_nao_opiniao)

        prob_opiniao_dado_frase =  np.prod(prob_op)*prior_op
        prob_nao_opiniao_dado_frase = np.prod(prob_nop)*prior_nop

    if prob_opiniao_dado_frase > prob_nao_opiniao_dado_frase:
        resultado.append(1)
    else:
        resultado.append(0)


test["resultados"] = resultado




In [157]:
pd.crosstab(test["Classificacao"],test["resultados"])


resultados,0,1
Classificacao,,
0,162,31
1,21,5
2,10,2
3,19,0


In [158]:
pd.crosstab(test['Classificacao'],test['resultados'],normalize="index")

resultados,0,1
Classificacao,,
0,0.839378,0.160622
1,0.807692,0.192308
2,0.833333,0.166667
3,1.000000,0.000000


In [160]:
lista_teste = [i for i in test["Classificacao"]]
lista_results = [i for i in test["resultados"]]
contador = 0
for i in range(len(lista_teste)):
    if lista_teste[i] == lista_results[i]:
        contador += 1
acertos = contador/len(lista_teste)
acertos *= 100
print(acertos)

66.8


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**